In [1]:
from dask.distributed import Client, LocalCluster
import bokeh
import geopandas as gpd
import scipy
import libpysal
import momepy
import json
from dask import delayed
import glob
import dask
import dask.dataframe as dd
import momepy
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import json
from tqdm import tqdm
from shapely.geometry import Point
import networkx as nx

/tmp/ipykernel_3271/2859145375.py:3: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
daskCluster = LocalCluster(threads_per_worker=2,
                n_workers=8, memory_limit='100GB')

client = Client(daskCluster)

client

2023-04-18 05:56:12,705 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-18 05:56:12,707 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-18 05:56:12,710 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-18 05:56:12,712 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-18 05:56:12,715 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-18 05:56:12,718 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-18 05:56:12,720 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-18 05:56:12,

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 16,Total memory: 250.47 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:41815,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 250.47 GiB
Comm: tcp://127.0.0.1:43909,Total threads: 2
Dashboard: http://127.0.0.1:44879/status,Memory: 31.31 GiB
Nanny: tcp://127.0.0.1:36743,


In [3]:
with open('./out/singapore/what_cells_are_in_what_cluster_dict.json', 'r') as f:
    includes_dict = json.load(f)
    
tessellation = gpd.read_parquet("./out/singapore/tessellation.pq")

buildings = gpd.read_parquet("./out/singapore/buildings.pq")

streets = gpd.read_parquet("./out/singapore/streets.pq").explode()

/tmp/ipykernel_3271/2172364374.py:8: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  streets = gpd.read_parquet("./out/singapore/streets.pq").explode()


In [4]:
streets['nID'] = momepy.unique_id(streets)
# buildings['nID'] = momepy.get_network_id(buildings, streets,
#                                                'nID').values

# buildings = buildings.drop_duplicates("uID").drop(columns="index_right")
# tessellation = tessellation.merge(buildings[['uID', 'nID']], on='uID', how='left')

In [5]:
### Distance Weight
dist200 = libpysal.weights.DistanceBand.from_dataframe(buildings, 200, ids='uID')

/home/virgilxw/miniconda3/lib/python3.10/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 520 disconnected components.
 There are 195 islands with ids: 99, 113, 921, 1567, 1722, 1724, 2523, 2536, 2559, 2562, 2566, 2612, 2619, 2630, 3467, 3476, 3482, 3510, 3513, 3514, 3515, 3516, 3524, 4055, 4177, 4178, 4188, 4189, 4238, 4670, 4819, 4820, 4947, 5488, 5502, 5611, 5639, 5651, 5672, 5676, 6193, 6322, 6436, 6576, 7127, 7131, 7439, 7440, 7449, 7540, 8379, 8380, 8741, 8960, 9128, 9215, 9216, 9220, 9221, 9222, 9283, 9292, 10240, 12427, 12978, 13708, 14446, 15110, 15172, 15173, 15176, 16024, 16029, 16677, 16694, 16695, 17122, 17419, 17960, 18648, 18666, 18692, 21202, 22013, 22669, 22670, 24504, 24514, 24515, 25723, 26327, 26343, 29028, 29033, 29037, 29038, 29891, 31014, 32389, 32390, 32566, 33077, 33157, 33935, 34204, 34473, 34920, 35640, 36329, 41504, 42003, 42592, 44147, 44304, 44305, 44744, 45136, 45137, 45151, 45584, 45587, 4

In [6]:
@dask.delayed
def queen_contiguity_load(n1, includes_dict):
    chunk = gpd.read_parquet(f"./out/singapore/expanded_cells_in_chunk_{int(n1)}.pq")
    
    # weights
    sw1 = libpysal.weights.w_subset(momepy.sw_high(k=1, gdf=chunk, ids='uID'), includes_dict[str(n1)]).to_adjlist()
    sw3 = libpysal.weights.w_subset(momepy.sw_high(k=3, gdf=chunk, ids='uID'), includes_dict[str(n1)]).to_adjlist()
    
    return (sw1, sw3)

In [7]:
file_pattern = "./out/singapore/expanded_cells_in_chunk_*.pq"
file_list = glob.glob(file_pattern)
num_files = len(file_list)

# weight_out = [process(n1, includes_dict) for n1 in tqdm(range(num_files))]

weight_out = [queen_contiguity_load(n1, includes_dict) for n1 in range(num_files)]

weight_out = dask.compute(weight_out)


<string>:1: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
<string>:1: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in y

In [8]:
sw1 = libpysal.weights.W.from_adjlist(pd.concat(weight_out[0][i][0] for i in range(num_files)))

sw3 = libpysal.weights.W.from_adjlist(pd.concat(weight_out[0][i][1] for i in range(num_files)))

/home/virgilxw/miniconda3/lib/python3.10/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 67 disconnected components.
  warnings.warn(message)


In [9]:
# # Find the uIDs in tessellation that are not in buildings
# tessellation_uids = set(tessellation['uID'])
# buildings_uids = set(buildings['uID'])
# new_uids = tessellation_uids - buildings_uids

# # Create a new GeoDataFrame with the new uIDs and empty geometry
# new_rows = pd.DataFrame({'uID': list(new_uids)})
# new_rows['geometry'] = None
# new_gdf = gpd.GeoDataFrame(new_rows, crs=buildings.crs)

# # Append the new GeoDataFrame to buildings
# new_buildings = pd.concat([buildings, new_gdf], ignore_index=True)

In [10]:
### Building Area
buildings['area'] = momepy.Area(buildings).series

## Building Shapes

### Circular Compactness
buildings['circular_com'] =  momepy.CircularCompactness(buildings).series

### Elongation
blg_elongation = momepy.Elongation(buildings)
buildings['elongation'] = blg_elongation.series

## Squareness
blg_squareness = momepy.Squareness(buildings)
buildings['squareness'] = blg_squareness.series

buildings['eri'] = momepy.EquivalentRectangularIndex(buildings).series

buildings['area'] = momepy.Area(buildings).series


buildings['orientation'] = momepy.Orientation(buildings).series

  0%|          | 0/76533 [00:00<?, ?it/s]

/home/virgilxw/miniconda3/lib/python3.10/site-packages/momepy/shape.py:962: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if geom.type == "Polygon":
/home/virgilxw/miniconda3/lib/python3.10/site-packages/momepy/shape.py:956: RuntimeWarning: invalid value encountered in arccos
  angle = np.degrees(np.arccos(cosine_angle))
/home/virgilxw/miniconda3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/virgilxw/miniconda3/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


  0%|          | 0/76533 [00:00<?, ?it/s]

/home/virgilxw/miniconda3/lib/python3.10/site-packages/momepy/distribution.py:74: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if geom.type in ["Polygon", "MultiPolygon", "LinearRing"]:


In [11]:
tessellation["convexity"] = momepy.Convexity(tessellation).series
tessellation["area"] = momepy.Area(tessellation).series

tessellation["neighbours"] = momepy.Neighbors(tessellation, sw3,'uID').series
tessellation["neighbours400"] = momepy.Neighbors(tessellation, dist200, 'uID').series
tessellation["covered_area"] = momepy.CoveredArea(tessellation, sw1, "uID", verbose=False).series

areas = momepy.Area(tessellation).series
tessellation["mean_area"] = momepy.AverageCharacter(
tessellation, values=areas, spatial_weights=sw3, unique_id='uID').mean

tessellation["mean_area_iq"] = momepy.AverageCharacter(
    tessellation, areas, sw3, 'uID', rng=(25, 75)).mean

tessellation["area_rng"] = momepy.Range(tessellation, values='area',
                                        spatial_weights=sw3, unique_id='uID').series
tessellation["area_iqr"] = momepy.Range(tessellation, values='area',
                        spatial_weights=sw3, unique_id='uID',
                        rng=(25, 75)).series
tessellation["rea_theil"] = momepy.Theil(tessellation, values='area',
                        spatial_weights=sw3,
                        unique_id='uID').series

tessellation["shannon"] = momepy.Shannon(tessellation, values='area',spatial_weights=sw3, unique_id='uID').series


  0%|          | 0/81870 [00:00<?, ?it/s]

  0%|          | 0/81870 [00:00<?, ?it/s]

  0%|          | 0/81870 [00:00<?, ?it/s]

  0%|          | 0/81870 [00:00<?, ?it/s]

  0%|          | 0/81870 [00:00<?, ?it/s]

  0%|          | 0/81870 [00:00<?, ?it/s]

  0%|          | 0/81870 [00:00<?, ?it/s]

  0%|          | 0/81870 [00:00<?, ?it/s]

In [12]:
edg_lin = momepy.Linearity(streets)
streets['linearity'] = edg_lin.series

profile = momepy.StreetProfile(streets, buildings)
streets["width"] = profile.w
streets["width_deviation"] = profile.wd
streets["openness"] = profile.o

/home/virgilxw/miniconda3/lib/python3.10/site-packages/momepy/shape.py:1286: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if geom.type == "LineString"


In [13]:
# primal = momepy.gdf_to_nx(streets, approach='primal')

In [14]:
%%time
# momepy.node_degree(primal)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.01 µs


In [15]:
%%time
# primal = momepy.closeness_centrality(primal, radius=400, name='closeness400', distance='mm_len', weight='mm_len')

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs


In [16]:
%%time
# primal = momepy.meshedness(primal, radius=400, distance="mm_len")

CPU times: user 0 ns, sys: 2 µs, total: 2 µs
Wall time: 4.77 µs


In [17]:
# %%time
# primal = momepy.straightness_centrality(primal)

In [18]:
# %%time
# primal = momepy.closeness_centrality(primal, name='closeness_global', weight='mm_len')

In [19]:
# %%time
# primal =  momepy.betweenness_centrality(primal, name='betweenness_metric_n', mode='nodes', weight='mm_len', k=100)

In [20]:
# %%time
# primal = momepy.straightness_centrality(primal)
# # momepy.mean_nodes(primal, 'straightness')
# momepy.mean_nodes(primal, 'closeness400')
# # momepy.mean_nodes(primal, 'closeness_global')
# # momepy.mean_nodes(primal, 'betweenness_metric_n')

# nodes, streets = momepy.nx_to_gdf(primal)

In [21]:
# dual = momepy.gdf_to_nx(streets, approach='dual')

# dual = momepy.closeness_centrality(dual, radius=5, name='angcloseness400', weight='angle')

# # dual = momepy.closeness_centrality(dual, name='closeness_global_ang', weight='angle')

# # dual = momepy.betweenness_centrality(dual, name='angbetweenness', mode='nodes', weight='angle')

# edges_d = momepy.nx_to_gdf(dual)


In [22]:
# Find the uIDs in tessellation that are not in buildings
tessellation_uids = set(tessellation['uID'])
buildings_uids = set(buildings['uID'])
new_uids = tessellation_uids - buildings_uids

# Create a new GeoDataFrame with the new uIDs and empty geometry
new_rows = pd.DataFrame({'uID': list(new_uids)})
new_rows['geometry'] = None
new_gdf = gpd.GeoDataFrame(new_rows, crs=buildings.crs)

# Append the new GeoDataFrame to buildings
buildings = pd.concat([buildings, new_gdf], ignore_index=True)

In [23]:
buildings.head()

,geometry,uID,area,circular_com,elongation,squareness,eri,orientation
0,"POLYGON ((4098.105 23706.883, 4109.145 23706.8...",0.0,135.936426,0.632538,0.896277,0.028693,1.000012,0.004613
1,"POLYGON ((4169.087 23688.953, 4169.269 23731.3...",1.0,940.600377,0.523132,0.523543,0.012337,1.000018,0.245322
2,"POLYGON ((4129.786 23795.994, 4196.953 23795.9...",2.0,3326.535481,0.608176,0.737375,0.000006,1.000000,0.004602
3,"POLYGON ((3940.171 23898.114, 3939.741 23952.6...",3.0,1412.582668,0.493066,0.474393,0.012235,1.000018,0.451083
4,"POLYGON ((3908.968 23952.996, 3934.866 23953.1...",4.0,1412.739688,0.493120,0.474631,0.009556,0.999984,0.451164


In [24]:
circular_compactness = momepy.CircularCompactness(buildings)
buildings['weighted_circom'] = momepy.WeightedCharacter(
buildings, circular_compactness.series, sw3, 'uID', momepy.Area(buildings).series).series

buildings['neighbour_dist'] = momepy.NeighborDistance(buildings, sw1, 'uID').series

buildings['interbuilding_distance'] = momepy.MeanInterbuildingDistance(buildings, sw1, 'uID', 3, verbose=False).series

interblg_distance = momepy.MeanInterbuildingDistance(
buildings, sw1, 'uID', spatial_weights_higher=sw3)
buildings['mean_ib_dist'] = interblg_distance.series

buildings['mean_ib_dist'] = momepy.MeanInterbuildingDistance(
    buildings, sw1, 'uID', order=3).series




  0%|          | 0/83366 [00:00<?, ?it/s]

  0%|          | 0/83366 [00:00<?, ?it/s]

/home/virgilxw/miniconda3/lib/python3.10/site-packages/momepy/distribution.py:658: RuntimeWarning: Mean of empty slice
  np.nanmean([x[-1] for x in list(sub.edges.data("weight"))])


Computing mean interbuilding distances...


  0%|          | 0/83366 [00:00<?, ?it/s]

Computing mean interbuilding distances...


  0%|          | 0/83366 [00:00<?, ?it/s]

In [25]:
cols = list(streets.columns)
cols.remove('nID')
cols.remove('geometry')

In [26]:
buildings['weighted_circom'] = momepy.WeightedCharacter(
buildings, circular_compactness.series, sw3, 'uID', momepy.Area(buildings).series).series

  0%|          | 0/83366 [00:00<?, ?it/s]

In [27]:
buildings = buildings.sjoin_nearest(streets.drop(cols,axis=1).reset_index(drop=True), max_distance=1000, how="left")

/home/virgilxw/miniconda3/lib/python3.10/site-packages/geopandas/geodataframe.py:2173: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:3414

  return geopandas.sjoin_nearest(


In [28]:
buildings = buildings.dropna(subset=['geometry'])
tessellation = tessellation.dropna(subset=['geometry'])

In [29]:
buildings['orientation'] = momepy.Orientation(buildings).series
tessellation['orientation'] = momepy.Orientation(tessellation).series

  0%|          | 0/81756 [00:00<?, ?it/s]

/home/virgilxw/miniconda3/lib/python3.10/site-packages/momepy/distribution.py:74: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if geom.type in ["Polygon", "MultiPolygon", "LinearRing"]:


  0%|          | 0/81870 [00:00<?, ?it/s]

In [30]:
str_align = momepy.StreetAlignment(buildings, streets,
                                   'orientation', 'nID',
                                   'nID')
buildings['str_align'] = str_align.series

/home/virgilxw/miniconda3/lib/python3.10/site-packages/momepy/distribution.py:84: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  elif geom.type in ["LineString", "MultiLineString"]:


In [31]:
blg_cell_align = momepy.CellAlignment(buildings, tessellation,
                                      'orientation', 'orientation',
                                      'uID', 'uID')
buildings['cell_align'] = blg_cell_align.series

In [32]:
tessellation['car'] = momepy.AreaRatio(tessellation, buildings, 'area', 'area', 'uID').series

In [33]:
nodes = gpd.read_parquet("./out/singapore/vertices_frame.pq")
edges = gpd.read_parquet("./out/singapore/edges_frame.pq")

In [34]:
tessellation.merge(buildings.drop(columns=['nID', 'geometry']), on='uID')

,uID,eID,geometry,cluster,convexity,area_x,neighbours,neighbours400,covered_area,mean_area,...,squareness,eri,orientation_y,weighted_circom,neighbour_dist,interbuilding_distance,mean_ib_dist,index_right,str_align,cell_align
0,73486.0,5.0,"POLYGON ((42345.292 33839.768, 42345.607 33840...",3,0.886686,1.564864e+05,37.0,2.0,319585.300120,47082.068839,...,0.383896,1.000544,44.403936,NaN,35.982106,146.619244,146.619244,21535.0,14.427933,20.196621
1,73508.0,5.0,"POLYGON ((42205.213 33723.253, 42205.300 33724...",3,0.915413,9.755781e+04,38.0,3.0,540704.593267,49738.761128,...,0.371892,1.000167,43.452823,NaN,221.412526,127.927147,127.927147,21535.0,13.476819,30.249631
2,73517.0,5.0,"POLYGON ((42013.570 34030.610, 42013.300 34031...",3,0.997869,4.647755e+04,31.0,10.0,535165.085971,45005.451528,...,0.363621,1.000426,32.322665,NaN,179.198136,115.325400,115.325400,46514.0,0.104342,3.944007
3,73520.0,5.0,"POLYGON ((42033.449 34502.816, 42033.834 34503...",3,0.980130,8.155060e+04,49.0,3.0,683422.139846,31737.009861,...,0.368506,1.000508,37.899454,NaN,204.655629,111.683092,111.683092,6258.0,22.424421,3.623193
4,73519.0,5.0,"POLYGON ((42253.669 34181.953, 42253.369 34182...",3,0.985041,2.401828e+05,44.0,0.0,708029.675710,35939.982520,...,0.416586,0.888889,40.200152,NaN,360.155519,93.811879,93.811879,6271.0,11.168597,32.140868
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79617,13930.0,9911.0,"POLYGON ((19895.841 33207.977, 19908.684 33221...",6,0.954012,2.580386e+03,39.0,10.0,38476.101908,5038.288381,...,42.426533,1.001068,1.991313,NaN,45.085465,57.791850,57.791850,77383.0,40.844483,42.075559
79618,36329.0,11046.0,"POLYGON ((27847.387 22035.537, 27847.387 22125...",6,0.961406,3.511055e+04,NaN,0.0,NaN,NaN,...,0.362146,1.000442,39.165931,NaN,NaN,NaN,NaN,NaN,NaN,39.165902
79619,31010.0,11051.0,"POLYGON ((26729.305 26689.564, 26729.305 26692...",6,0.875194,2.553828e+02,NaN,18.0,NaN,NaN,...,0.278649,1.000249,21.502767,NaN,NaN,NaN,NaN,66920.0,8.510991,23.262957
79620,29594.0,11054.0,"POLYGON ((25956.732 32902.381, 25957.801 32904...",6,0.977315,5.028825e+02,NaN,6.0,NaN,NaN,...,53.968256,1.065516,19.725148,NaN,NaN,NaN,NaN,32232.0,11.844089,0.000000


In [39]:
nodes

,vertID,betweeness_centrality,geometry,node_degree,closeness,global_closeness_centrality,global_clustering_400m,eigenvector,node_centrality_400m
0,0.0,0.000602,POINT (34430.050 43637.987),3,0.000000,0.000049,0.333333,0.0,8
1,1.0,0.001453,POINT (34208.291 43724.424),3,0.000000,0.000049,0.250000,45394.0,10
2,2.0,0.001522,POINT (34233.732 43841.503),4,0.000000,0.000049,0.000000,45395.0,8
3,3.0,0.000464,POINT (33868.362 43252.963),3,0.075907,0.000050,0.096774,1.0,14
4,4.0,0.000447,POINT (33626.442 43076.363),3,0.370707,0.000049,0.000000,6252.0,4
...,...,...,...,...,...,...,...,...,...
68300,68300.0,0.000000,POINT (45440.187 23228.149),3,0.000000,0.000059,0.222222,62630.0,8
68301,68301.0,0.000093,POINT (45706.851 23146.849),3,0.210423,0.000060,0.089552,62631.0,24
68302,68302.0,0.000000,POINT (38041.687 17877.535),1,0.026989,0.000065,0.129032,62633.0,57
68303,68303.0,0.000000,POINT (38069.332 17842.346),1,0.031639,0.000065,0.133333,62634.0,55


In [44]:
tessellation = tessellation.sjoin(nodes, how="left")
tessellation = tessellation.sjoin(edges, how="left")

ValueError: 'index_left' and 'index_right' cannot be names in the frames being joined

In [49]:
tessellation.columns.tolist()

['uID',
 'eID',
 'geometry',
 'cluster',
 'convexity',
 'area',
 'neighbours',
 'neighbours400',
 'covered_area',
 'mean_area',
 'mean_area_iq',
 'area_rng',
 'area_iqr',
 'rea_theil',
 'shannon',
 'orientation',
 'car',
 'index_right',
 'vertID',
 'betweeness_centrality',
 'node_degree',
 'closeness',
 'global_closeness_centrality',
 'global_clustering_400m',
 'eigenvector',
 'node_centrality_400m']

In [55]:
tessellation = tessellation.drop(['vertID','index_right'],axis=1)

,uID,eID,geometry,cluster,convexity,area,neighbours,neighbours400,covered_area,mean_area,...,shannon,orientation,car,betweeness_centrality,node_degree,closeness,global_closeness_centrality,global_clustering_400m,eigenvector,node_centrality_400m
0,73486.0,5.0,"POLYGON ((42345.292 33839.768, 42345.607 33840...",3,0.886686,156486.443851,37.0,2.0,319585.300120,47082.068839,...,1.416431,24.207316,0.003469,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,73508.0,5.0,"POLYGON ((42205.213 33723.253, 42205.300 33724...",3,0.915413,97557.813503,38.0,3.0,540704.593267,49738.761128,...,1.384372,13.203191,0.057234,0.000539,3.0,0.037797,0.000063,0.0,21396.0,36.0
6,73508.0,5.0,"POLYGON ((42205.213 33723.253, 42205.300 33724...",3,0.915413,97557.813503,38.0,3.0,540704.593267,49738.761128,...,1.384372,13.203191,0.057234,0.000510,2.0,0.051346,0.000063,0.0,53375.0,36.0
6,73508.0,5.0,"POLYGON ((42205.213 33723.253, 42205.300 33724...",3,0.915413,97557.813503,38.0,3.0,540704.593267,49738.761128,...,1.384372,13.203191,0.057234,0.000481,2.0,0.058967,0.000063,0.0,53336.0,37.0
6,73508.0,5.0,"POLYGON ((42205.213 33723.253, 42205.300 33724...",3,0.915413,97557.813503,38.0,3.0,540704.593267,49738.761128,...,1.384372,13.203191,0.057234,0.000452,2.0,0.044496,0.000063,0.0,53337.0,37.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85201,2006828.0,NaN,"POLYGON ((6081.374 33262.202, 6077.044 33266.3...",0,1.000000,106.591112,NaN,NaN,NaN,NaN,...,NaN,44.011977,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85202,2006829.0,NaN,"POLYGON ((4363.226 30071.240, 4362.933 30071.2...",0,1.000000,3.285191,NaN,NaN,NaN,NaN,...,NaN,41.665750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85203,2006830.0,NaN,"POLYGON ((3636.078 26623.923, 3635.651 26845.7...",0,0.179129,8542.694616,NaN,NaN,NaN,NaN,...,NaN,2.837112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85204,2006831.0,NaN,"POLYGON ((3180.754 25467.087, 3038.080 25466.6...",0,0.189062,3966.492010,NaN,NaN,NaN,NaN,...,NaN,0.143549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
# merged.to_parquet("./out/singapore/.pq")